In [3]:
import json
from pathlib import Path
from typing import Any, Dict, List, Tuple


# ---------- 基礎格式工具 ----------

def _join_lines(lines: List[str]) -> str:
    return "<br>".join([l for l in lines if l is not None and l != ""])


def _fmt_step_cell(step_part: Dict[str, Any]) -> str:
    """
    step_part: INPUT / PROCESS / OUTPUT
    - INPUT 必填，但這裡仍以資料存在為準
    - PROCESS/OUTPUT 若 present=false 則視為空白
    - text + sub_steps 會合併在同一格
    """
    if not step_part:
        return ""

    if step_part.get("present") is False:
        return ""

    text = (step_part.get("text") or "").strip()
    subs = step_part.get("sub_steps") or []

    if subs:
        sub_lines = []
        for s in subs:
            sub_no = (s.get("sub_no") or "").strip()
            sub_text = (s.get("text") or "").strip()
            if sub_no or sub_text:
                sub_lines.append(f"&nbsp;&nbsp;{sub_no} {sub_text}".strip())
        if text:
            return _join_lines([text] + sub_lines)
        return _join_lines(sub_lines)

    return text


def _step_sort_key(step_no: str) -> Tuple[int, int, str]:
    """
    讓 step_no 合理排序：❶,❷,... 再到 ❶-①,❶-②...
    """
    circled_map = {"❶": 1, "❷": 2, "❸": 3, "❹": 4, "❺": 5, "❻": 6, "❼": 7, "❽": 8, "❾": 9, "❿": 10}
    sub_map = {"①": 1, "②": 2, "③": 3, "④": 4, "⑤": 5, "⑥": 6, "⑦": 7, "⑧": 8, "⑨": 9, "⑩": 10}

    step_no = (step_no or "").strip()
    if not step_no:
        return (999, 999, "")

    if "-" in step_no:
        parent, sub = step_no.split("-", 1)
        return (circled_map.get(parent, 999), sub_map.get(sub, 999), step_no)

    return (circled_map.get(step_no, 999), 0, step_no)


# ---------- 轉換：JSON -> Markdown ----------

def json_to_three_column_sdd_table(d: Dict[str, Any]) -> str:
    """
    真正三欄 SDD 表：
      | INPUT | PROCESS | OUTPUT |
      每個 step 一列
    """
    steps = d.get("步驟表") or []
    steps_sorted = sorted(steps, key=lambda st: _step_sort_key(st.get("step_no", "")))

    lines: List[str] = []
    lines.append("| INPUT | PROCESS | OUTPUT |")
    lines.append("|---|---|---|")

    for st in steps_sorted:
        step_no = (st.get("step_no") or "").strip()

        inp = _fmt_step_cell(st.get("INPUT") or {})
        proc = _fmt_step_cell(st.get("PROCESS") or {})
        outp = _fmt_step_cell(st.get("OUTPUT") or {})

        # 每格前面加 step_no，貼近你們 Word 範本呈現
        inp_cell = (f"{step_no} {inp}".strip() if inp else "").replace("\n", "<br>")
        proc_cell = (f"{step_no} {proc}".strip() if proc else "").replace("\n", "<br>")
        out_cell = (f"{step_no} {outp}".strip() if outp else "").replace("\n", "<br>")

        lines.append(f"| {inp_cell} | {proc_cell} | {out_cell} |")

    return "\n".join(lines)


def json_to_md_document(d: Dict[str, Any]) -> str:
    """
    產生完整 md（含標頭資訊 + 三欄表 + 資料表欄位）
    你如果只想要三欄表，可以把標頭區塊註解掉。
    """
    fn_id = d.get("功能編號", "")
    fn_name = d.get("功能名稱", "")
    fn_desc = d.get("功能說明", "")
    prog = d.get("程式名稱", "")

    in_sum = d.get("輸入摘要") or {}
    out_sum = d.get("輸出摘要") or {}

    in_trigger = "、".join(in_sum.get("觸發名詞") or [])
    in_res = "、".join(in_sum.get("資源名詞") or [])
    out_trigger = "、".join(out_sum.get("觸發名詞") or [])
    out_res = "、".join(out_sum.get("資源名詞") or [])

    note = d.get("備註", "")

    # 資料表欄位（若有）
    table = d.get("資料表") or {}
    tname = table.get("table_name")
    cols = table.get("columns") or []

    md: List[str] = []

    # --- 標頭資訊（你們 Word 常會有） ---
    md.append(f"# {fn_id} {fn_name}".strip())
    md.append("")
    md.append(f"- **功能編號**：{fn_id}")
    md.append(f"- **功能名稱**：{fn_name}")
    md.append(f"- **功能說明**：{fn_desc}")
    md.append(f"- **程式名稱**：{prog}")
    md.append(f"- **輸入**：{in_trigger}" + (f"；{in_res}" if in_res else ""))
    md.append(f"- **輸出**：{out_trigger}" + (f"；{out_res}" if out_res else ""))
    if note:
        md.append(f"- **備註**：{note}")
    md.append("")

    # --- 三欄 SDD 表 ---
    md.append("## INPUT / PROCESS / OUTPUT")
    md.append("")
    md.append(json_to_three_column_sdd_table(d))
    md.append("")

    # --- 資料表（若有） ---
    if tname:
        md.append("## 資料表")
        md.append("")
        md.append(f"### {tname}")
        md.append("")
        md.append("| 欄位 | 中文 | 型態 | 長度 | 空值 | 預設值 | 備註 |")
        md.append("|---|---|---|---:|---|---|---|")
        for c in cols:
            md.append(
                f"| {c.get('欄位','')} | {c.get('中文','')} | {c.get('型態','')} | "
                f"{'' if c.get('長度') is None else c.get('長度')} | {c.get('空值','')} | "
                f"{'' if c.get('預設值') is None else c.get('預設值')} | "
                f"{'' if c.get('備註') is None else c.get('備註')} |"
            )
        md.append("")

    return "\n".join(md)


# ---------- 批次轉檔 ----------

def build_fmd(
    input_dir: str = r"design\SDD\功能描述",
    output_dir: str = r"design\SDD\FMD"
) -> None:
    in_path = Path(input_dir)
    out_path = Path(output_dir)

    if not in_path.exists():
        raise FileNotFoundError(f"找不到輸入資料夾：{in_path.resolve()}")

    out_path.mkdir(parents=True, exist_ok=True)

    json_files = sorted(in_path.glob("*.json"))
    if not json_files:
        print(f"⚠️ 沒有找到任何 .json：{in_path.resolve()}")
        return

    ok, fail = 0, 0
    for jf in json_files:
        try:
            with jf.open("r", encoding="utf-8") as f:
                data = json.load(f)

            md_text = json_to_md_document(data)

            out_file = out_path / (jf.stem + ".md")
            out_file.write_text(md_text, encoding="utf-8")

            ok += 1
            print(f"✅ {jf.name} -> {out_file.name}")

        except Exception as e:
            fail += 1
            print(f"❌ {jf.name} 轉換失敗：{e}")

    print("")
    print(f"完成：成功 {ok} 份，失敗 {fail} 份")
    print(f"輸出資料夾：{out_path.resolve()}")


if __name__ == "__main__":
    build_fmd()


✅ 1_1.json -> 1_1.md
✅ 1_1_1.json -> 1_1_1.md
✅ 1_1_2.json -> 1_1_2.md
✅ 1_2.json -> 1_2.md
✅ 1_2_1.json -> 1_2_1.md
✅ 1_3.json -> 1_3.md
✅ 1_3_1.json -> 1_3_1.md
✅ 1_3_2.json -> 1_3_2.md
✅ 2_1.json -> 2_1.md
✅ 2_1_1.json -> 2_1_1.md
✅ 2_1_2.json -> 2_1_2.md
✅ 2_1_3.json -> 2_1_3.md
✅ 2_1_4.json -> 2_1_4.md
✅ 2_1_5.json -> 2_1_5.md
✅ 2_1_6.json -> 2_1_6.md
✅ 2_2.json -> 2_2.md
✅ 2_2_1.json -> 2_2_1.md
✅ 2_2_2.json -> 2_2_2.md
✅ 2_2_3.json -> 2_2_3.md
✅ 2_2_4.json -> 2_2_4.md
✅ 2_2_5.json -> 2_2_5.md
✅ 2_2_6.json -> 2_2_6.md
✅ 2_2_7.json -> 2_2_7.md
✅ 2_2_8.json -> 2_2_8.md
✅ 2_3.json -> 2_3.md
✅ 2_3_1.json -> 2_3_1.md
✅ 2_3_2.json -> 2_3_2.md
✅ 2_3_3.json -> 2_3_3.md
✅ 2_3_4.json -> 2_3_4.md
✅ 2_3_6.json -> 2_3_6.md
✅ 2_3_7.json -> 2_3_7.md
✅ 2_4.json -> 2_4.md
✅ 2_4_1.json -> 2_4_1.md
✅ 2_4_2.json -> 2_4_2.md
✅ 2_4_3.json -> 2_4_3.md
✅ 2_4_4.json -> 2_4_4.md
✅ 2_5.json -> 2_5.md
✅ 2_5_1.json -> 2_5_1.md
✅ 2_5_2.json -> 2_5_2.md
✅ 2_5_3.json -> 2_5_3.md
✅ 2_5_4.json -> 2_5_4.md
✅ 2_6.j